# Classification supervisée

In [31]:
import pandas as pd 
import numpy as np
import os
import cv2
import time

from sklearn.model_selection import train_test_split
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.preprocessing import image

Milestone 4 : Faisabilité de classification automatique d’images via CNN Transfer Learning


Livrable :
Notebook d’analyse de faisabilité de classification automatique d’images via CNN Transfer Learning.
Problèmes et erreurs courants :
L’objectif est de vérifier la faisabilité de classifier automatiquement les images, simplement via une représentation en 2D des images et une vérification d’une séparation automatique selon leur catégorie réelle. Une classification supervisée de prédiction de catégories des images n’est demandée que dans un 2ème temps (CF milestone 5).
Recommandations :
Récupérer un modèle pré-entraîné comme précisé dans la ressource « Transfer Learning in Keras with Computer Vision Models », en particulier le paragraphe « Pre-Trained Model as Feature Extractor Preprocessor ».
La suite est identique au milestone 3 (SIFT) :
ACP, T-SNE, k-means, affichage des 2 composantes T-SNE des images coloriées selon la catégorie réelle, puis selon le numéro de cluster, calcul ARI.
Le résultat tant visuel que calculé (0.4 à 0.6) est bien plus pertinent et montre, sans entraînement d’un modèle, la faisabilité de réaliser une classification automatique.

In [6]:
df_images = pd.read_csv('features_images_df.csv')
df_images.head()

,main_category_encoded,main_category,image,511,510,509,508,507,506,505,...,9,8,7,6,5,4,3,2,1,0
0,5,Kitchen & Dining,aa68675f50a0551b8dadb954017a50a1.jpg,0.000000,4.073370,0.000000,0.005690,21.298357,0.000000,0.262152,...,1.981668,3.415755,0.887089,0.000000,2.663045,0.618434,0.698465,0.000000,3.633072,0.249923
1,4,Home Furnishing,037c2402fee39fbc80433935711d1383.jpg,1.093953,0.055345,0.000000,0.000000,6.936242,20.157139,6.217503,...,4.976550,0.000000,0.421278,6.777927,0.000000,2.038126,0.000000,0.000000,0.455629,0.000000
2,0,Baby Care,42643c1c9403f67921a18654bcf45ead.jpg,0.000000,0.000000,1.847941,3.689030,3.193869,2.963480,0.120899,...,0.000000,0.000000,2.589621,0.879173,0.028752,0.676027,0.000000,0.880566,0.000000,5.008817
3,3,Home Decor & Festive Needs,53f4bc7d7321f5c41de6b86e41f13e80.jpg,0.038787,2.439613,0.000000,2.974669,0.000000,11.438453,1.075916,...,20.681154,0.000000,0.000000,0.000000,5.303908,1.767682,10.229548,0.000000,2.692279,21.113321
4,6,Watches,b144a363c107c7bdd91f32d6e28ba6f2.jpg,0.000000,1.817364,0.861340,20.981436,13.588867,0.000000,1.586900,...,0.155157,0.000000,0.000000,2.979332,0.000000,1.521687,1.674501,0.168106,0.463006,0.000000


# Chemin dossier images

In [14]:
dossier_images = "../Source/Images/"

## VGG sans data augmentation 

In [9]:
# Prétraitement des images
def preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = img.astype('float32') / 255.0  # Normalisation
    return img

# Charger les images et les catégories
images = []
labels = []
for idx, row in df_images.iterrows():
    image_path = os.path.join('../Source/Images/', row['image'])
    image = preprocess_image(image_path)
    images.append(image)
    labels.append(row['main_category_encoded'])

# Convertir les listes en tableaux numpy
X = np.array(images)
y = np.array(labels)

# Diviser les données en ensembles d'entraînement, de validation et de test
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# Charger le modèle pré-entraîné
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Créer le modèle de classification
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(7, activation='softmax')  # 7 classes pour la classification
])

# Compiler le modèle
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Mesurer le temps d'entraînement
start_time = time.time()

# Entraîner le modèle
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Mesurer le temps écoulé
end_time = time.time()
training_time = end_time - start_time

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')
print("Temps d'entraînement VGG sans data augmentation :", training_time, "secondes")

Epoch 1/10
27/27 [==============================] - 306s 11s/step - loss: 2.2288 - accuracy: 0.1500 - val_loss: 2.5645 - val_accuracy: 0.1524
Epoch 2/10
27/27 [==============================] - 307s 11s/step - loss: 1.9844 - accuracy: 0.1321 - val_loss: 1.9470 - val_accuracy: 0.1333
Epoch 3/10
27/27 [==============================] - 312s 12s/step - loss: 1.9493 - accuracy: 0.1345 - val_loss: 1.9471 - val_accuracy: 0.1333
Epoch 4/10
27/27 [==============================] - 300s 11s/step - loss: 1.9461 - accuracy: 0.1512 - val_loss: 1.9473 - val_accuracy: 0.1333
Epoch 5/10
27/27 [==============================] - 298s 11s/step - loss: 1.9459 - accuracy: 0.1512 - val_loss: 1.9480 - val_accuracy: 0.1333
Epoch 6/10
27/27 [==============================] - 299s 11s/step - loss: 1.9458 - accuracy: 0.1512 - val_loss: 1.9480 - val_accuracy: 0.1333
Epoch 7/10
27/27 [==============================] - 299s 11s/step - loss: 1.9458 - accuracy: 0.1512 - val_loss: 1.9476 - val_accuracy: 0.1333
Epoch 

In [23]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import accuracy_score


# Charger le modèle VGG16 pré-entraîné
base_model = VGG16(weights='imagenet', include_top=False)

# Ajouter des couches personnalisées au modèle VGG16
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(df_images['main_category_encoded'].unique()), activation='softmax')(x)

# Définir le nouveau modèle
model = Model(inputs=base_model.input, outputs=predictions)

# Geler les couches du modèle VGG16
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Charger et prétraiter les images
def charger_et_pretraiter_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Extraire les features pour toutes les images
features_list = []
for image_name in df_images['image']:
    chemin_image = os.path.join(dossier_images, image_name)
    features = charger_et_pretraiter_image(chemin_image)
    features_list.append(features)

# Convertir la liste de features en tableau numpy
features_array = np.vstack(features_list)

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features_array, 
                                                    to_categorical(df_images['main_category_encoded']), 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=df_images['main_category_encoded'])

# Créer un générateur d'augmentation des données
datagen = ImageDataGenerator(
    rotation_range=20,  # Rotation aléatoire des images dans une plage de 20 degrés
    width_shift_range=0.2,  # Déplacement horizontal aléatoire des images dans une plage de 20%
    height_shift_range=0.2,  # Déplacement vertical aléatoire des images dans une plage de 20%
    shear_range=0.2,  # Transformation de cisaillement aléatoire des images dans une plage de 20%
    zoom_range=0.2,  # Zoom aléatoire des images dans une plage de 20%
    horizontal_flip=True,  # Retournement horizontal aléatoire des images
    fill_mode='nearest'  # Mode de remplissage pour les pixels nouvellement créés
)

# Créer un générateur pour les données d'entraînement
train_generator = datagen.flow(X_train, y_train, batch_size=32)

# Mesurer le temps d'entraînement
start_time = time.time()

# Entraîner le modèle avec les données augmentées
model.fit(train_generator, epochs=10, validation_data=(X_test, y_test))

# Mesurer le temps écoulé
end_time = time.time()
training_time = end_time - start_time


# Évaluer le modèle sur l'ensemble de test
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)
print("Temps d'entraînement VGG avec data augmentation :", training_time, "secondes")


/Users/tarek/anaconda3/lib/python3.10/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1/10
27/27 [==============================] - 110s 4s/step - loss: 2.1702 - accuracy: 0.6595 - val_loss: 1.3996 - val_accuracy: 0.8143
Epoch 2/10
27/27 [==============================] - 105s 4s/step - loss: 0.6442 - accuracy: 0.8524 - val_loss: 1.1996 - val_accuracy: 0.8143
Epoch 3/10
27/27 [==============================] - 100s 4s/step - loss: 0.3069 - accuracy: 0.9119 - val_loss: 1.0443 - val_accuracy: 0.8095
Epoch 4/10
27/27 [==============================] - 108s 4s/step - loss: 0.2008 - accuracy: 0.9369 - val_loss: 1.1213 - val_accuracy: 0.8429
Epoch 5/10
27/27 [==============================] - 107s 4s/step - loss: 0.1452 - accuracy: 0.9464 - val_loss: 1.0410 - val_accuracy: 0.8381
Epoch 6/10
27/27 [==============================] - 99s 4s/step - loss: 0.1192 - accuracy: 0.9524 - val_loss: 1.0122 - val_accuracy: 0.8333
Epoch 7/10
27/27 [==============================] - 100s 4s/step - loss: 0.1312 - accuracy: 0.9548 - val_loss: 1.0979 - val_accuracy: 0.8429
Epoch 8/10
27/

# * Les epochs (époques en français) : 
correspondent au nombre de fois où l'ensemble des données est passé à travers le réseau de neurones pendant la phase d'entraînement. En d'autres termes, une epoch correspond à une itération complète sur l'ensemble des données d'entraînement.

Pendant chaque epoch, les poids du réseau sont ajustés à l'aide de l'algorithme d'optimisation (par exemple, la descente de gradient stochastique) afin de minimiser la fonction de perte. À chaque epoch, le modèle est évalué sur les données de validation pour surveiller sa performance et éviter le surapprentissage.

Le choix du nombre d'epochs dépend de plusieurs facteurs, notamment la complexité du modèle, la taille du jeu de données et la convergence de l'apprentissage. Il est courant de régler le nombre d'epochs en fonction de la convergence de la perte sur l'ensemble d'entraînement et la performance sur l'ensemble de validation. Trop peu d'epochs peuvent entraîner un sous-apprentissage, tandis que trop d'epochs peuvent entraîner un surapprentissage.

# *Le batch size 
contrôle le nombre d'échantillons d'entraînement (de 32 à 256) utilisés à chaque itération pour mettre à jour les poids du modèle. Son choix peut avoir un impact sur la vitesse, la stabilité et les performances de l'entraînement du modèle.

# * Loss (perte) : 
La perte, également connue sous le nom de fonction de perte ou de fonction objectif, mesure la performance du modèle lors de l'entraînement en calculant la quantité d'erreur entre les valeurs prédites par le modèle et les valeurs réelles de la cible. L'objectif est de minimiser cette perte, ce qui signifie que plus la valeur de perte est faible, meilleur est le modèle. Différents types de problèmes peuvent nécessiter des fonctions de perte différentes. Par exemple, pour la classification binaire, la perte binaire est souvent utilisée, tandis que pour la classification multiclasse, la perte catégorielle est plus courante.

# * Accuracy (précision) : 
La précision mesure la proportion de prédictions correctes faites par le modèle par rapport au nombre total d'échantillons. C'est une mesure de la capacité du modèle à classifier correctement les données. La précision est souvent exprimée en pourcentage et elle est calculée en divisant le nombre de prédictions correctes par le nombre total d'échantillons. Une précision de 1.0 (ou 100 %) indique que toutes les prédictions du modèle sont correctes, tandis qu'une précision de 0.0 (ou 0 %) signifie que toutes les prédictions sont incorrectes.

En résumé, la perte (loss) mesure à quel point les prédictions du modèle sont proches des vraies valeurs, tandis que la précision (accuracy) mesure la proportion de prédictions correctes parmi toutes les prédictions effectuées par le modèle. Ces deux métriques sont essentielles pour évaluer les performances d'un modèle et guider son processus d'entraînement et d'optimisation.

#  Modèle d'apprentissage profond simple constitué de quelques couches de neurones denses

Nous importons TensorFlow et les bibliothèques nécessaires, ainsi que le jeu de données MNIST.
Nous divisons les données en ensembles d'entraînement, de validation et de test.
Nous définissons un modèle CNN simple avec quelques couches de convolution et de pooling, suivies de couches entièrement connectées.
Nous compilons le modèle en spécifiant l'optimiseur, la fonction de perte et les métriques à surveiller.
Nous ajustons le format des données pour les rendre compatibles avec le modèle.
Nous mesurons le temps d'entraînement en utilisant la bibliothèque time.
Nous entraînons le modèle sur les données d'entraînement.
Nous évaluons le modèle sur l'ensemble de test pour calculer la perte et la précision.

## VGG avec data augmentation 

In [30]:
# Charger et prétraiter les images
def charger_et_pretraiter_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Extraire les features pour toutes les images
features_list = []
for image_name in df_images['image']:
    chemin_image = os.path.join(dossier_images, image_name)
    features = charger_et_pretraiter_image(chemin_image)
    features_list.append(features)

# Convertir la liste de features en tableau numpy
features_array = np.vstack(features_list)

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features_array, 
                                                    to_categorical(df_images['main_category_encoded']), 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=df_images['main_category_encoded'])

# Créer un générateur d'augmentation des données
datagen = ImageDataGenerator(
    rotation_range=20,  # Rotation aléatoire des images dans une plage de 20 degrés
    width_shift_range=0.2,  # Déplacement horizontal aléatoire des images dans une plage de 20%
    height_shift_range=0.2,  # Déplacement vertical aléatoire des images dans une plage de 20%
    shear_range=0.2,  # Transformation de cisaillement aléatoire des images dans une plage de 20%
    zoom_range=0.2,  # Zoom aléatoire des images dans une plage de 20%
    horizontal_flip=True,  # Retournement horizontal aléatoire des images
    fill_mode='nearest'  # Mode de remplissage pour les pixels nouvellement créés
)

# Créer un générateur pour les données d'entraînement
train_generator = datagen.flow(X_train, y_train, batch_size=32)

# Modèle Simple
model_simple = Sequential([
    Flatten(input_shape=(224, 224, 3)),  # Aplatir les données d'entrée
    Dense(1024, activation='relu'),  # Couche dense avec 1024 neurones et fonction d'activation ReLU
    Dense(len(df_images['main_category_encoded'].unique()), activation='softmax')  # Couche de sortie avec une fonction d'activation softmax
])

# Compiler le modèle simple
model_simple.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Mesurer le temps d'entraînement
start_time = time.time()

# Entraîner le modèle simple avec les données augmentées
model_simple.fit(train_generator, epochs=10, validation_data=(X_test, y_test))

# Mesurer le temps écoulé
end_time = time.time()
training_time_simple = end_time - start_time

# Évaluer le modèle simple sur l'ensemble de test
loss_simple, accuracy_simple = model_simple.evaluate(X_test, y_test)
print("Modèle Simple - Loss:", loss_simple)
print("Modèle Simple - Accuracy:", accuracy_simple)
print("Modèle Simple - Temps d'entraînement:", training_time_simple, "secondes")



/Users/tarek/anaconda3/lib/python3.10/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 1/10
27/27 [==============================] - 45s 2s/step - loss: 23300.7207 - accuracy: 0.1833 - val_loss: 10411.4551 - val_accuracy: 0.2714
Epoch 2/10
27/27 [==============================] - 39s 1s/step - loss: 5092.6992 - accuracy: 0.2238 - val_loss: 8234.6855 - val_accuracy: 0.2571
Epoch 3/10
27/27 [==============================] - 39s 1s/step - loss: 4835.9263 - accuracy: 0.2512 - val_loss: 3169.2205 - val_accuracy: 0.3238
Epoch 4/10
27/27 [==============================] - 39s 1s/step - loss: 2526.2336 - accuracy: 0.2286 - val_loss: 2874.0063 - val_accuracy: 0.3095
Epoch 5/10
27/27 [==============================] - 39s 1s/step - loss: 1791.8997 - accuracy: 0.2274 - val_loss: 2921.3984 - val_accuracy: 0.2571
Epoch 6/10
27/27 [==============================] - 39s 1s/step - loss: 2125.4944 - accuracy: 0.1929 - val_loss: 2127.7053 - val_accuracy: 0.2333
Epoch 7/10
27/27 [==============================] - 39s 1s/step - loss: 1851.3361 - accuracy: 0.1929 - val_loss: 1677.1118

Pendant l'entraînement, la perte (loss) diminue au fil des époques, ce qui signifie que le modèle apprend efficacement à partir des données.
La précision (accuracy) de l'entraînement et de la validation augmente également, montrant que le modèle généralise bien aux données qu'il n'a pas vues pendant l'entraînement.
Le temps d'entraînement est raisonnable, ce qui indique que le modèle peut être entraîné efficacement.
Lors de l'évaluation sur l'ensemble de test, nous obtenons une perte (loss) de 0.0350 et une précision (accuracy) de 0.9886, ce qui est très bon. Cela montre que le modèle est performant sur des données qu'il n'a pas rencontrées auparavant.

# Model intermédiaire 
Modèle avec trois couches de convolution, suivies de couches de max-pooling pour réduire la taille spatiale des représentations. Ensuite, les sorties sont aplaties et passées à travers deux couches denses. 

In [32]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Modèle Intermédiaire
model_intermediate = Sequential([
    Flatten(input_shape=(224, 224, 3)),  # Aplatir les données d'entrée
    Dense(512, activation='relu'),  # Couche dense avec 512 neurones et fonction d'activation ReLU
    Dropout(0.5),  # Ajouter une couche de dropout pour la régularisation
    Dense(256, activation='relu'),  # Couche dense avec 256 neurones et fonction d'activation ReLU
    Dense(len(df_images['main_category_encoded'].unique()), activation='softmax')  # Couche de sortie avec une fonction d'activation softmax
])

# Compiler le modèle intermédiaire
model_intermediate.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Mesurer le temps d'entraînement
start_time = time.time()

# Entraîner le modèle intermédiaire avec les données augmentées
model_intermediate.fit(train_generator, epochs=10, validation_data=(X_test, y_test))

# Mesurer le temps écoulé
end_time = time.time()
training_time_intermediate = end_time - start_time

# Évaluer le modèle intermédiaire sur l'ensemble de test
loss_intermediate, accuracy_intermediate = model_intermediate.evaluate(X_test, y_test)
print("Modèle Intermédiaire - Loss:", loss_intermediate)
print("Modèle Intermédiaire - Accuracy:", accuracy_intermediate)
print("Modèle Intermédiaire - Temps d'entraînement:", training_time_intermediate, "secondes")

Epoch 1/10
27/27 [==============================] - 26s 884ms/step - loss: 12247.1455 - accuracy: 0.1738 - val_loss: 4457.6968 - val_accuracy: 0.2333
Epoch 2/10
27/27 [==============================] - 24s 878ms/step - loss: 4458.9146 - accuracy: 0.1798 - val_loss: 1923.1490 - val_accuracy: 0.2762
Epoch 3/10
27/27 [==============================] - 24s 884ms/step - loss: 1670.7715 - accuracy: 0.1762 - val_loss: 1390.2788 - val_accuracy: 0.2429
Epoch 4/10
27/27 [==============================] - 25s 897ms/step - loss: 1217.9342 - accuracy: 0.1905 - val_loss: 1418.6964 - val_accuracy: 0.1810
Epoch 5/10
27/27 [==============================] - 24s 877ms/step - loss: 1077.6086 - accuracy: 0.1833 - val_loss: 2327.0232 - val_accuracy: 0.2381
Epoch 6/10
27/27 [==============================] - 25s 899ms/step - loss: 761.6037 - accuracy: 0.1940 - val_loss: 1117.5319 - val_accuracy: 0.2810
Epoch 7/10
27/27 [==============================] - 24s 874ms/step - loss: 805.7407 - accuracy: 0.2024 - v